In [1]:
import logging
LOG = logging.getLogger("probas")
logging.basicConfig(level=logging.ERROR)
logging.disable(logging.INFO)
import json
from pprint import pprint
from itertools import chain

import requests
from transformers import BertTokenizer, BertForMaskedLM
import torch
from tqdm.auto import tqdm

# Add parent dir, so we can access our common code
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

# from mlyoucanuse.bert_fun import (
#     get_alternate_words, 
#     get_word_probabilities,
#     get_word_in_sentence_probability, 
#     sum_log_probabilities)

/cephfs/home/kravtsova/hse/thesis/ML-You-Can-Use/ml_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/cephfs/home/kravtsova/hse/thesis/ML-You-Can-Use/ml_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/cephfs/home/kravtsova/hse/thesis/ML-You-Can-Use/ml_venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('/home/kravtsova/rubert_cased_L-12_H-768_A-12_pt/vocab.txt', do_lower_case=False)
bert_model = BertForMaskedLM.from_pretrained('/home/kravtsova/rubert_cased_L-12_H-768_A-12_pt',
                                             config="/home/kravtsova/rubert_cased_L-12_H-768_A-12_pt/bert_config.json")


In [3]:
bert_model = bert_model.to('cuda')
bert_model.device

device(type='cuda', index=0)

In [45]:
def sentence_prob(
    sentence: str, bert_model: BertForMaskedLM, bert_tokenizer: BertTokenizer, batch_size: int = 32
) -> float:
    device = bert_model.device
    
    positions = []
    subtokens = []
    
    offset = 0
    for token in sentence.split():
        st = bert_tokenizer.encode(token, add_special_tokens=False)
        positions.append((offset, offset + len(st)))
        offset += len(st)
        subtokens.extend(st)
    
    start_token_id = [bert_tokenizer.cls_token_id]
    end_token_id = [bert_tokenizer.sep_token_id]
    
    if len(subtokens) > 510:
        LOG.warning("Too many tokens, should be 510 or less, found %s", total_tokens)
    softmax = torch.nn.Softmax(dim=2).to(device=device)
    
    bert_model.eval()
    
    full_data = [start_token_id
             + subtokens[:start]
             + [bert_tokenizer.mask_token_id]*(end-start)
             + subtokens[end:] + end_token_id
             for start, end in positions]
    
    res = []
    with torch.no_grad():
        for batch_start in range(0, len(full_data), batch_size):
            batch = torch.tensor(full_data[batch_start:batch_start+batch_size], device=device)
            outputs = bert_model(batch, token_type_ids=batch * 0)
            predictions = softmax(outputs[0]) * 100
            predictions = predictions.log().cpu().numpy()
            
            for row, (start, end) in zip(predictions, positions[batch_start:batch_start+batch_size]):
                for i in range(start, end):
                    res.append(row[i+1][subtokens[i]])
    
    return sum(res)

In [6]:
from typing import Tuple, List, Dict

def get_alternate_words(
    sentence: str,
    word_index: int,
    bert_model: BertForMaskedLM,
    bert_tokenizer: BertTokenizer,
    top: int = 10,
) -> Tuple[Tuple[str, float]]:
    
    device = bert_model.device
    whole_tokens = sentence.split()
    bert_token_map = {
        idx: bert_tokenizer.encode(whole_token, add_special_tokens=False)
        for idx, whole_token in enumerate(whole_tokens)
    }
    bert_token_map[word_index] = bert_tokenizer.encode(
        "[MASK]", add_special_tokens=False
    )
    total_tokens = len(list(chain.from_iterable(bert_token_map.values())))
    LOG.debug("total bert tokens: %s whole tokens: %s", total_tokens, len(whole_tokens))
    torch.set_grad_enabled(False)

    start_token_id = bert_tokenizer.encode("[CLS]", add_special_tokens=False)
    end_token_id = bert_tokenizer.encode("[SEP]", add_special_tokens=False)
    the_tokens = list(chain.from_iterable(bert_token_map.values()))
    LOG.debug(bert_tokenizer.convert_ids_to_tokens(the_tokens))
    indexed_tokens = list(start_token_id) + the_tokens + list(end_token_id)
    softmax = torch.nn.Softmax(dim=1).to(device=device)
    with torch.no_grad():
        # pylint: disable=not-callable,no-member
        tokens_tensor = torch.tensor([indexed_tokens]).to(device=device)
        segments_tensors = torch.tensor(
            [
                torch.zeros(  # type: ignore
                    len(indexed_tokens), dtype=int
                ).tolist()
            ]
        ).to(device=device)
        outputs = bert_model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = softmax(outputs[0].squeeze(0))
    # to find the true index of the desired word; count all of the tokens and subtokens before
    the_word_idx = (
        len(
            list(
                chain.from_iterable(
                    [vals for key, vals in bert_token_map.items() if key < word_index]
                )
            )
        )
        + 1
    )
    probas, indices = torch.topk(  # pylint: disable=no-member
        predictions[the_word_idx], top
    )
    alt_words = bert_tokenizer.convert_ids_to_tokens(indices.cpu().tolist())
    return tuple(zip(alt_words, probas.cpu().tolist()))  # type: ignore

In [7]:
get_alternate_words(sentence='Я иду на школу', 
                    word_index=2,
                    bert_tokenizer=bert_tokenizer,
                    bert_model=bert_model,
                    top=10)

(('в', 0.9766497611999512),
 ('за', 0.003301926888525486),
 ('на', 0.0021766619756817818),
 ('смотреть', 0.00120127375703305),
 ('через', 0.0004980001249350607),
 ('музыкальную', 0.0004563286202028394),
 ('посмотреть', 0.00041629490442574024),
 ('во', 0.0003552580892574042),
 ('новую', 0.00034798699198290706),
 ('работать', 0.0003149556287098676))

#### Оставить/убрать запятую (проверка вероятности рубертом)

In [8]:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct.txt', 'r', encoding='utf8'
         ) as f:
    test = f.readlines()
sents = [x.strip() for x in test]

In [9]:
for j,sent in tqdm(enumerate(sents)):
    if ',' in sent:
        spl = sent.split()
        for i, tok in enumerate(spl):
            orig_prob = sentence_prob(sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
    #         print('Orig sent: {}'.format(sent))
    #         print('Orig sent prob: {}'.format(orig_prob))
            if tok == ',':
                tmp = sent.split()
                del tmp[i]
                new_sent = ' '.join(tmp)
                new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
        #             print('New sent: {}'.format(new_sent))
        #             print('New sent prob: {}'.format(new_prob))
                thr = 10
                if new_prob > (orig_prob + thr):
                    sents[j] = sent = new_sent
#                     print(new_sent)
    print(sent)
#     with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase.txt', 'a', encoding='utf8') as f:
#         print(sent, file=f, flush=True)

Хотя природные явления ( извержения , цветения водорослей , землетрясения , и т .
Экологи показывали страшные фотографии и дали неопределенные угрозы , которые могут происходить на АЭС до или после строительства .
плакат А . Николаева , за который арестовали его в 2012
Работал с 1966 до 2008
Общая площадь : 68 кв .
В связи с тем , что эра Сталина произошло вместе со становлении Магадана , Магадан также стал частью механизмы террора и тирании сталинской власти .
Какие плюсы и минусы существуют ? ( Независимость , гордость своей страной , возвращение национальной культуры / нереальность обучения на этих языках , падание от большого русско говоряшего мира , современность русского языка , и . т . д . )



#### Маскирование однословного предлога рубертом

In [11]:
# with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase.txt', 'r', encoding='utf8') as f:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct.txt', 'r', encoding='utf8') as f:
    test_orig = f.readlines()
test_orig = [x.strip() for x in test_orig]

In [10]:
with open('russian_preps.txt', 'r') as f:
    rus_preps = f.readlines()
rus_preps = {x.strip() for x in rus_preps}

In [13]:
# with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_parsed.txt',
#          'r') as f:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct_parsed.txt', 'r') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [14]:
indices = []
for sent in test_parsed:
    inds = []
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'ADP') and (tok[7] == 'case'):
            start = i
            i += 1
            while i < len(synt_info) and synt_info[i][7] == 'fixed':
                i += 1
            prep = ' '.join([x[1] for x in synt_info[start:i]]).lower()
            if prep in rus_preps:
                inds.append(start)
        i += 1
    indices.append(inds)

In [16]:
# с удалением предлога
for j,(sent,ind) in tqdm(enumerate(zip(test_orig[4504:], indices[4504:]))):
    removed = 0
    for i in ind:
        i -= removed
        sub = get_alternate_words(sentence=sent, 
                    word_index=i,
                    bert_tokenizer=bert_tokenizer,
                    bert_model=bert_model,
                    top=1)[0][0]
        probs = []
        tokens = sent.split()
        no_prep = ' '.join(tokens[:i] + tokens[i+1:])
        no_prep_prob = sentence_prob(no_prep, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
        probs.append((no_prep_prob, no_prep, 1))
        orig_prob = sentence_prob(sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
        if sub.lower() in rus_preps and sub != sent.split()[i]:
            tmp = sent.split()
            tmp[i] = sub
            new_sent = ' '.join(tmp)
            new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
            probs.append((new_prob, new_sent, 0))
        thr = 10
        max_prob, max_sent, new_removed = max(probs)
        if max_prob > (orig_prob + thr):
            print('Orig sent: {}'.format(sent))
            print('Orig sent prob: {}'.format(orig_prob))
            print('New sent: {}'.format(max_sent))
            print('New sent prob: {}'.format(max_prob))
#             sent = max_sent
            test_orig[j] = sent = max_sent
            removed += new_removed
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked-deleted.txt', 'a', encoding='utf8') as f:
        print(sent, file=f, flush=True)

Orig sent: Ради этого я жила в России четыре месяца и планирую вернуться туда на год .
Orig sent prob: 25.459430158138275
New sent: До этого я жила в России четыре месяца и планирую вернуться туда на год .
New sent prob: 35.542846247553825
Orig sent: Чхартишвили окончил историко-филологическое отделение Института стран Азии и Африки в МГУ .
Orig sent prob: -12.113512516021729
New sent: Чхартишвили окончил историко-филологическое отделение Института стран Азии и Африки при МГУ .
New sent prob: 4.087726026773453
Orig sent: Даже перед того , как видела эту сцену , я знала , что должно происходить .
Orig sent prob: 30.641615748405457
New sent: Даже после того , как видела эту сцену , я знала , что должно происходить .
New sent prob: 54.77656734455377
Orig sent: Во времени войны было сложно путешествовать в восточной Европе .
Orig sent prob: 7.343004658818245
New sent: Ко времени войны было сложно путешествовать в восточной Европе .
New sent prob: 19.72600269317627
Orig sent: Хотя многие бы

In [22]:
# без удаления предлога
for j,(sent,ind) in tqdm(enumerate(zip(test_orig, indices))):
    for i in ind:
        sub = get_alternate_words(sentence=sent, 
                    word_index=i,
                    bert_tokenizer=bert_tokenizer,
                    bert_model=bert_model,
                    top=1)[0][0]
        if sub.lower() in rus_preps and sub != sent.split()[i]:
            orig_prob = sentence_prob(sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
            tmp = sent.split()
            tmp[i] = sub
            new_sent = ' '.join(tmp)
            new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
            thr = 10
            if new_prob > (orig_prob + thr):
                print('Orig sent: {}'.format(sent))
                print('Orig sent prob: {}'.format(orig_prob))
                print('New sent: {}'.format(new_sent))
                print('New sent prob: {}'.format(new_prob))
                test_orig[j] = sent = new_sent
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked.txt', 'a', encoding='utf8') as f:
        print(sent, file=f, flush=True)

Orig sent: Потому что они постоянно находятся в меняющемся рынке , они редко застигнуты врасплох .
Orig sent prob: 9.066727340221405
New sent: Потому что они постоянно находятся на меняющемся рынке , они редко застигнуты врасплох .
New sent prob: 27.873316407203674
Orig sent: Можно спорить с идей , что Россия западная страна , но пока Россия стремится к западному стандарту жизни , следует согласиться , что стандарты , способы и термины будет похожие .
Orig sent prob: 36.78791615366936
New sent: Можно спорить насчет идей , что Россия западная страна , но пока Россия стремится к западному стандарту жизни , следует согласиться , что стандарты , способы и термины будет похожие .
New sent prob: 46.82103311084211
Orig sent: Леонтьев не только работал в Америке ; он как-то успел влюбиться на красивую американскую поэтессу Эстель Маркс .
Orig sent prob: 2.015134032815695
New sent: Леонтьев не только работал в Америке ; он как-то успел влюбиться в красивую американскую поэтессу Эстель Маркс .
N

#### Перебор словоформ существительных из предложных групп рубертом (skills_2)

In [16]:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_parsed.txt',
         'r') as f:
    test_ya_parsed = f.read()
sents = test_ya_parsed.split('\n\n')
del sents[-1]

In [ ]:
for sent in tqdm(sents):
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    orig_sent = ' '.join(x[1] for x in synt_info)
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'ADP') and (tok[7] == 'case') and (synt_info[int(tok[6])-1][3] == 'NOUN'):
            lexem = morph.parse(synt_info[int(tok[6])-1][2])
            try:
                lex = [x for x in lexem if 'NOUN' in x.tag and 'nomn' in x.tag][0]
            except IndexError:
                print('IndexError')
                print(orig_sent)
                i += 1
                continue
            hyp = {l.word for l in lex.lexeme if l.tag.POS == lex.tag.POS}
            orig_prob = sentence_prob(orig_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model,
                                                  batch_size=16)
            new_probs = dict()
            for h in hyp:
                if synt_info[int(tok[6])-1][0].isupper():
                    h = h.capitalize()
                tmp = orig_sent.split()
                tmp[int(synt_info[int(tok[6])-1][0]) - 1] = h
                new_sent = ' '.join(tmp)
                new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
                new_probs[str(new_prob)] = new_sent
            thr = 10
            new_dict = {k:v for (k,v) in new_probs.items() if float(k) > (orig_prob + thr)}
            if not new_dict:
                i += 1
                continue
            max_item = max(new_dict.items(), key=lambda x: float(x[0]))
            orig_sent = max_item[1]
            print(orig_sent)
        i += 1
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked_prep-masked_noun-masked-test.txt', 'a', encoding='utf8') as f:
        print(orig_sent, file=f, flush=True)

В частности , в 2086 году .
Есть возможность , что представителей Гуманитарного Действия приедут в город Вашингтон летом и , лучше всего , в Портленд в ноябре ! Я сказала , что я готова переводить для них и работать экскурсоводом , если на самом деле приедут в нашей замечательный город .
Потом я устроился на работу , и всё было хорошо .
Тамилой рассказывает Пете о своем напитке от всех злах и страданий , о своем кольце с жабой , которое ее бережет , о птице Сирин - птица смерти , о птице Финист , и Алконост , и дарит Пете волшебное редкое яйцо Алконоста .
Первостепенное доказательство того , что эти красноармейцы – бандиты , появляется рано в поэме , к концу части 2 , где Блок описывает их словами : « В зубах – цыгарка , примят картуз , / / На спинах б надо бубновый туз ! » .
В процессе этого эксперимента руководители и финансовые лидеры проекта проводят другие реформы , которые решают проблемы с коррупцией и привлекают иностранных специалистов или собственных , отечественных талантов 

### Исправление по фреймбанку с перебором рубертом

In [21]:
import pandas as pd
from itertools import chain
from collections import defaultdict
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [22]:
frames = pd.read_csv('framebank_dict_cx_items.txt', sep='\t')

In [24]:
# ranks = set(frames['Rank'])
# forms = list(set(frames['Form']))
grams = {'Sacc': ['NOUN', 'Case=Acc'],
        'Sdat': ['NOUN', 'Case=Dat'],
        'Sgen': ['NOUN', 'Case=Gen'],
        'Sins': ['NOUN', 'Case=Ins'],
        'Sloc': ['NOUN', 'Case=Loc'],
        'Snom': ['NOUN', 'Case=Nom']}

In [25]:
new_frame = frames.groupby('ConstrIndex', as_index=False).agg({'Form': list,
                                                               'Rank': list,
                                                   'KeyLexemes': 'first'}).reset_index()

In [26]:
new_frame[new_frame['KeyLexemes'].str.contains("угрожать")]
# test_fr = new_frame[new_frame['KeyLexemes'].str.contains("угрожать")]['Form'].tolist()
# test_lx = new_frame[new_frame['KeyLexemes'].str.contains("угрожать")]['KeyLexemes'].tolist()

,index,ConstrIndex,Form,Rank,KeyLexemes
10766,10766,11924,"[Snom, угрожать, Sdat, Sins]","[Субъект, Предикат, Периферия, Периферия]",угрожать
10767,10767,11925,"[Snom, угрожать, что + CL]","[Субъект, Предикат, Клауза]",угрожать
10768,10768,11926,"[Snom, угрожать, Sdat]","[Субъект, Предикат, Периферия]",угрожать
10769,10769,11927,"[Snom, угрожать, Sdat, Sins]","[Субъект, Предикат, Периферия, Периферия]",угрожать
10770,10770,11928,"[Snom, угрожать, Vinf]","[Субъект, Предикат, Периферия]",угрожать
10771,10771,11929,"[Snom, угрожать, Sdat]","[Субъект, Предикат, Периферия]",угрожать
10857,10857,12028,"[Snom, угрожать]","[Субъект, Предикат]",угрожать
12438,12438,13862,"[Snom, угрожать, Sdat]","[Субъект, Предикат, Периферия]",угрожать
12439,12439,13863,"[Snom, угрожать, Vinf]","[Субъект, Предикат, Периферия]",угрожать
12440,12440,13864,"[Snom, угрожать, Sdat]","[Субъект, Предикат, Периферия]",угрожать


In [27]:
#new
def forms_and_roles(forms, roles):
    return [(x,y) for x,y in zip(forms, roles)]

In [30]:
#old (working)
def make_patterns(patterns):
    for i in range(len(patterns)):
        for j in range(len(patterns[i])):
            if '+' in patterns[i][j]:
                patterns[i][j] = patterns[i][j].split(' + ')
    l = []
    for item in patterns:
        new_item = []
        for word in item:
            if isinstance(word, str):
                new_item.append(word)
            else:
                new_item.extend(word)
        l.append(new_item)
    return l

In [28]:
dp2pymorphy = {'Nom':'nomn', 'Gen':'gent', 'Dat':'datv', 'Acc':'accs', 'Ins':'ablt', 'Loc':'loct', 'Voc':'voct',
               'Par':'gen2', 'Sing':'sing', 'Plur':'plur', 'Masc':'masc', 'Fem':'femn', 'Neut':'neut', 'Sup':'Supr',
                 'Anim':'anim', 'Inan':'inan', 'Short':'ADJS'}

In [29]:
disambigious_preps = {'на', 'в', 'за', 'с', 'через', 'со', 'для', 'под'}

In [30]:
obliq = ['Sgen', 'Sdat', 'Sacc', 'Sins', 'Sloc']

In [31]:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_parsed.txt',
         'r') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]
sents = test_parsed

In [56]:
#verb-noun case agreement for noun

for sent in tqdm(sents):
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    orig_sent = ' '.join(x[1] for x in synt_info)
    deps = [synt_info[i][0] for i in range(len(synt_info))]
    heads = [synt_info[i][6] for i in range(len(synt_info))]
    heads_deps = defaultdict(set)
    for h, d in zip(heads, deps):
        heads_deps[h].add(d)
    hd = dict(heads_deps)
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'VERB'):
            forms = new_frame[new_frame['KeyLexemes'].str.contains(tok[2])]['Form'].tolist()
            cases = list(set([x for x in list(chain.from_iterable(forms)) if x in obliq and x != 'Snom']))
            roles = new_frame[new_frame['KeyLexemes'].str.contains(tok[2])]['Rank'].tolist()
            fr = forms_and_roles(forms, roles)
            fr = [x for x in fr if any(y in x[0] for y in obliq)]
            if len(fr) > 0:
                try:
                    for j in hd[str(i+1)]:
                        if (synt_info[int(j)-1][3] == 'NOUN') and (
                            (synt_info[int(j)-1][7] == 'obj') or (synt_info[int(j)-1][7] == 'obl')) and (
                        (j not in hd) or (all((synt_info[int(ind)-1][3] != 'ADP') for ind in hd[j]))):
#                             print('Noun ind: ' + j)
#                             print('Noun: {}'.format(synt_info[int(j)-1][2]))
#                             print('Noun grams: {}'.format(synt_info[int(j)-1][5]))
#                             print('Noun role: {}'.format(synt_info[int(j)-1][7]))
                            frame_fs = list(set([grams[case][1] for case in cases]))
#                             print(frame_fs)
                            noun_fs = synt_info[int(j)-1][5].split('|')
                            common = [x for x in noun_fs if x in frame_fs]
                            if not common:
                                lexem = morph.parse(synt_info[int(j)-1][2])
                                try:
                                    lex = [x for x in lexem if 'NOUN' in x.tag and 'nomn' in x.tag][0]
                                except IndexError:
                                    print('IndexError')
                                    print(orig_sent)
                                    continue
                                hyp = {l.word for l in lex.lexeme if l.tag.POS == lex.tag.POS}
                                orig_prob = sentence_prob(orig_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model,
                                                  batch_size=16)
                                new_probs = dict()
                                for h in hyp:
                                    if synt_info[int(j)-1][1][0].isupper():
                                        h = h.capitalize()
                                    tmp = orig_sent.split()
                                    tmp[int(j)-1] = h
                                    new_sent = ' '.join(tmp)
                                    new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
                                    new_probs[str(new_prob)] = new_sent
                                thr = 10
                                new_dict = {k:v for (k,v) in new_probs.items() if float(k) > (orig_prob + thr)}
                                if not new_dict:
                                    continue
                                max_item = max(new_dict.items(), key=lambda x: float(x[0]))
                                orig_sent = max_item[1]
#                                 orig_prob = float(max_item[0])
                                print(orig_sent)
                except KeyError:
                    print('Нет такого индекса в словаре')
        i += 1
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun.txt', 'a', encoding='utf8') as f:
        print(orig_sent, file=f, flush=True)

Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Человека только нужно уметь чувствовать , и , смотря на картину или читая красивые стихи , его эмоциональный интеллект .
Нет такого индекса в словаре
Нет такого индекса в словаре
Хлеба будет много , голод остается досадам капитализма .
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Нет такого индекса в словаре
Они не сознает , почему это грозит их жизням .
Нет такого индекса в словаре
Нет такого индекса в 

#### Перебор словоформ прилагательных, зависимых от существительных, рубертом (skills_2)

In [33]:
# with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_parsed.txt',
#          'r') as f:
with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct_prep-masked_prep-noun-masked_frame-v+noun_parsed.txt',
         'r') as f:
    test_ya_parsed = f.read()
sents = test_ya_parsed.split('\n\n')
del sents[-1]

In [34]:
#excluded
for sent in tqdm(sents):
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    orig_sent = ' '.join(x[1] for x in synt_info)
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'ADJ') and (tok[7] == 'amod' or tok[7] == 'root') and (synt_info[int(tok[6])-1][3] == 'NOUN' or (
            tok[6] == '0')):
            lexem = morph.parse(tok[2])
            try:
                lex = [x for x in lexem if 'ADJF' in x.tag][0]
            except IndexError:
                print('IndexError')
                print(orig_sent)
                i += 1
                continue
            hyp = {l.word for l in lex.lexeme if l.tag.POS == lex.tag.POS}
            orig_prob = sentence_prob(orig_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model,
                                                  batch_size=16)
            new_probs = dict()
            for h in hyp:
                if tok[1][0].isupper():
                    h = h.capitalize()
                tmp = orig_sent.split()
                tmp[int(tok[0])-1] = h
                new_sent = ' '.join(tmp)
                new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
                new_probs[str(new_prob)] = new_sent
            thr = 10
            new_dict = {k:v for (k,v) in new_probs.items() if float(k) > (orig_prob + thr)}
            if not new_dict:
                i += 1
                continue
            max_item = max(new_dict.items(), key=lambda x: float(x[0]))
            orig_sent = max_item[1]
            print(orig_sent)
        i += 1
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/excluded_test_ya_speller_punct_prep-masked_prep-noun-masked_frame-v+noun_adj-masked.txt', 'a', encoding='utf8') as f:
        print(orig_sent, file=f, flush=True)

In [ ]:
for sent in tqdm(sents[356:]):
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    orig_sent = ' '.join(x[1] for x in synt_info)
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'ADJ') and (tok[7] == 'amod' or tok[7] == 'root') and (synt_info[int(tok[6])-1][3] == 'NOUN' or (
            tok[6] == '0')):
            lexem = morph.parse(tok[2])
            try:
                lex = [x for x in lexem if 'ADJF' in x.tag][0]
            except IndexError:
                print('IndexError')
                print(orig_sent)
                i += 1
                continue
            hyp = {l.word for l in lex.lexeme if l.tag.POS == lex.tag.POS}
            orig_prob = sentence_prob(orig_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model,
                                                  batch_size=16)
            new_probs = dict()
            for h in hyp:
                if tok[1][0].isupper():
                    h = h.capitalize()
                tmp = orig_sent.split()
                tmp[int(tok[0])-1] = h
                new_sent = ' '.join(tmp)
                new_prob = sentence_prob(new_sent, bert_tokenizer=bert_tokenizer, bert_model=bert_model)
                new_probs[str(new_prob)] = new_sent
            thr = 10
            new_dict = {k:v for (k,v) in new_probs.items() if float(k) > (orig_prob + thr)}
            if not new_dict:
                i += 1
                continue
            max_item = max(new_dict.items(), key=lambda x: float(x[0]))
            orig_sent = max_item[1]
            print(orig_sent)
        i += 1
    with open('/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset/skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_adj-masked.txt', 'a', encoding='utf8') as f:
        print(orig_sent, file=f, flush=True)

IndexError
В 1926-ом году знаменитая американская артистка цирка Мэрион Диксон приезжает на гастроли в Москву .
IndexError
Конечно , много эмигрантов покидает свою родину в развивающемся « Втором мире » или неразвитом « Третьем мире » в поисках лучших экономических условий , большей степени религиозной свободы или большей степени политической свободы в развитых странах « Первого мира » .
IndexError
Конечно , много эмигрантов покидает свою родину в развивающемся « Втором мире » или неразвитом « Третьем мире » в поисках лучших экономических условий , большей степени религиозной свободы или большей степени политической свободы в развитых странах « Первого мира » .
IndexError
В России , также как и в США , в течение двадцатых годов не думали , что скоро будет Великая депрессия .
IndexError
Я родился в семье - пятым ребенком из двенадцати детей .
IndexError
За вторыми – будущее .
IndexError
На первой фотографии стоит молодая женщина , которая одета в стиле 70-х годов .
IndexError
Я знаю , ч